In [ ]:
!pip install -q yt-dlp
import yt_dlp as ydl
import subprocess
from pathlib import Path

tiktok_url = "https://vt.tiktok.com/ZSaMwSq5s/"

ydl_opts = {
    "outtmpl": "tiktok.%(ext)s",
    # Try best video+audio, fallback to best combined, then any format with audio
    "format": "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/bestvideo+bestaudio/best",
    "merge_output_format": "mp4",
    "postprocessors": [{
        "key": "FFmpegVideoConvertor",
        "preferedformat": "mp4",
    }],
    # Ensure we get audio
    "prefer_ffmpeg": True,
    "keepvideo": False,
}

with ydl.YoutubeDL(ydl_opts) as y:
    info = y.extract_info(tiktok_url, download=True)
    video_path = y.prepare_filename(info)

# Verify audio stream exists
check_audio = subprocess.run(
    ["ffprobe", "-i", video_path, "-show_streams", "-select_streams", "a", "-loglevel", "error"],
    capture_output=True,
    text=True
)

if not check_audio.stdout.strip():
    raise ValueError(f"Downloaded video has no audio stream. TikTok URL may not contain audio: {tiktok_url}")

print(f"✓ Downloaded with audio: {video_path}")


[vm.tiktok] Extracting URL: https://vt.tiktok.com/ZSaMwSq5s/
[vm.tiktok] ZSaMwSq5s: Downloading webpage
[TikTok] Extracting URL: https://www.tiktok.com/@calebwritescode/video/7597188385920306445?_r=1&_t=ZS-93GwBoSyB6L
[TikTok] 7597188385920306445: Downloading webpage


[info] 7597188385920306445: Downloading 1 format(s): bytevc1_1080p_139582-1
[download] tiktok.mp4 has already been downloaded
[download] 100% of    6.32MiB
[VideoConvertor] Not converting media file "tiktok.mp4"; already is in target format mp4
✓ Downloaded with audio: tiktok.mp4


In [ ]:
# @title 🌴 Change the values in this section

# @markdown Select the source of the audio/video file to be transcribed
input_format = "local" #@param ["youtube", "gdrive", "local"]

# @markdown Enter the URL of the YouTube video or the path of the audio file to be transcribed
file = "tiktok.mp4" #@param {type:"string"}

#@markdown Click here if you'd like to save the transcription as text file
plain = True #@param {type:"boolean"}

# @markdown Click here if you'd like to save the transcription as an SRT file
srt = True #@param {type:"boolean"}

#@markdown Click here if you'd like to save the transcription as a VTT file
vtt = True #@param {type:"boolean"}

#@markdown Click here if you'd like to save the transcription as a TSV file
tsv = True #@param {type:"boolean"}

#@markdown Click here if you'd like to download the transcribed file(s) locally
download = True #@param {type:"boolean"}

In [ ]:
# Dependencies

!pip install -q pytube
!pip install -q git+https://github.com/openai/whisper.git

import os, re
import torch
from pathlib import Path
from pytube import YouTube

import whisper
from whisper.utils import get_writer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Use CUDA, if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load the desired model
model = whisper.load_model("medium.en").to(DEVICE)

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 115MiB/s]


In [ ]:
def transcribe_file(model, file, plain, srt, vtt, tsv, download):
    """
    Runs Whisper on an audio file

    Parameters
    ----------
    model: Whisper
        The Whisper model instance.

    file: str
        The file path of the file to be transcribed.

    plain: bool
        Whether to save the transcription as a text file or not.

    srt: bool
        Whether to save the transcription as an SRT file or not.

    vtt: bool
        Whether to save the transcription as a VTT file or not.

    tsv: bool
        Whether to save the transcription as a TSV file or not.

    download: bool
        Whether to download the transcribed file(s) or not.

    Returns
    -------
    A dictionary containing the resulting text ("text") and segment-level details ("segments"), and
    the spoken language ("language"), which is detected when `decode_options["language"]` is None.
    """
    file_path = Path(file)
    print("running function")
    print(f"Transcribing file: {file_path}\n")

    output_directory = file_path.parent

    # Run Whisper
    result = model.transcribe(file, verbose = False, language = "en")

    if plain:
        txt_path = file_path.with_suffix(".txt")
        print(f"\nCreating text file")

        with open(txt_path, "w", encoding="utf-8") as txt:
            txt.write(result["text"])
    if srt:
        print(f"\nCreating SRT file")
        srt_writer = get_writer("srt", output_directory)
        srt_writer(result, str(file_path.stem))

    if vtt:
        print(f"\nCreating VTT file")
        vtt_writer = get_writer("vtt", output_directory)
        vtt_writer(result, str(file_path.stem))

    if tsv:
        print(f"\nCreating TSV file")

        tsv_writer = get_writer("tsv", output_directory)
        tsv_writer(result, str(file_path.stem))

    if download:
        from google.colab import files

        colab_files = Path("/content")
        stem = file_path.stem

        for colab_file in colab_files.glob(f"{stem}*"):
            if colab_file.suffix in [".txt", ".srt", ".vtt", ".tsv"]:
                print(f"Downloading {colab_file}")
                files.download(str(colab_file))

    return result

In [ ]:
import os
print("cwd:", os.getcwd())
print(os.listdir)
#

if input_format == "youtube":
    # Download the audio stream of the YouTube video
    print(f"Downloading audio stream: {audio}")
    audio = download_youtube_audio(file)

    # Run Whisper on the audio stream
    result = transcribe_file(model, audio, plain, srt, vtt, tsv, download)
elif input_format == "gdrive":
    # Authorize a connection between Google Drive and Google Colab
    from google.colab import drive
    drive.mount('/content/drive')

    # Run Whisper on the specified file
    result = transcribe_file(model, file, plain, srt, vtt, tsv, download)
elif input_format == "local":
    import subprocess

    # Check if video has audio stream using ffprobe
    check_audio = subprocess.run(
        ["ffprobe", "-i", file, "-show_streams", "-select_streams", "a", "-loglevel", "error"],
        capture_output=True,
        text=True
    )

    if not check_audio.stdout.strip():
        raise ValueError(f"Video file '{file}' contains no audio stream. Cannot transcribe.")

    # Convert video file to audio file for transcription
    audio_file_name = Path(file).stem + ".wav"
    result = subprocess.run(
        ["ffmpeg", "-i", file, "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", "-y", audio_file_name],
        capture_output=True,
        text=True
    )

    if result.returncode != 0:
        raise RuntimeError(f"FFmpeg failed: {result.stderr}")

    # Verify the file was created
    if not Path(audio_file_name).exists():
        raise FileNotFoundError(f"Failed to create audio file: {audio_file_name}")

    # Run Whisper on the specified audio file
    result = transcribe_file(model, audio_file_name, plain, srt, vtt, tsv, download)


cwd: /content
<built-in function listdir>
running function
Transcribing file: tiktok.wav



100%|██████████| 37997/37997 [00:49<00:00, 774.94frames/s]


Creating text file

Creating SRT file

Creating VTT file

Creating TSV file


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>